In [102]:
from scipy.interpolate import make_interp_spline

import pandas as pd
import glob
from utils import convert_csv_to_graph
from network import get_network_stats
import networkx as nx
import matplotlib.pyplot as plt
import csv
import numpy as np
import seaborn as sns
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d
from utils import smooth
from data import get_MIM_df

from scipy.stats import kruskal


In [103]:
# Base case (w = 25, bfp = 0.15, b=2) 

#--------------------------------- w --------------------------------
mim1_w1_mdf, mim1_w1_adf = get_MIM_df('1_w=1')
mim1_w5_mdf, mim1_w5_adf = get_MIM_df('1_w=5')
mim1_w25_mdf, mim1_w25_adf = get_MIM_df('1_w=25')


mim2_w1_mdf,mim2_w1_adf = get_MIM_df('2_w=1')
mim2_w5_mdf,mim2_w5_adf = get_MIM_df('2_w=5')
mim2_w25_mdf,mim2_w25_adf = get_MIM_df('2_w=25')

mim3_w1_mdf,mim3_w1_adf = get_MIM_df('3_w=1')
mim3_w5_mdf,mim3_w5_adf = get_MIM_df('3_w=5')
mim3_w25_mdf,mim3_w25_adf = get_MIM_df('3_w=25')

#--------------------------------- bfp --------------------------------

mim1_bfp015_mdf, mim1_bfp015_adf = get_MIM_df('1_w=25')
mim2_bfp015_mdf, mim2_bfp015_adf = get_MIM_df('2_w=25')
mim3_bfp015_mdf, mim3_bfp015_adf = get_MIM_df('3_w=25')

mim1_bfp03_mdf, mim1_bfp03_adf = get_MIM_df('1_bfp=0.3')
mim2_bfp03_mdf, mim2_bfp03_adf = get_MIM_df('2_bfp=0.3')
mim3_bfp03_mdf, mim3_bfp03_adf = get_MIM_df('3_bfp=0.3')

mim1_bfp06_mdf, mim1_bfp06_adf = get_MIM_df('1_bfp=0.6')
mim2_bfp06_mdf, mim2_bfp06_adf = get_MIM_df('2_bfp=0.6')
mim3_bfp06_mdf, mim3_bfp06_adf = get_MIM_df('3_bfp=0.6')

#--------------------------------- b --------------------------------

mim1_b2_mdf, mim1_b2_adf = get_MIM_df('1_w=25')
mim2_b2_mdf, mim2_b2_adf = get_MIM_df('2_w=25')
mim3_b2_mdf, mim3_b2_adf = get_MIM_df('3_w=25')

mim1_b10_mdf, mim1_b10_adf = get_MIM_df('1_b=10')
mim2_b10_mdf, mim2_b10_adf = get_MIM_df('2_b=10')
mim3_b10_mdf, mim3_b10_adf = get_MIM_df('3_b=10')


num_agent = 1069



## Normality test

In [104]:
import scipy.stats as stats

# from pingouin import kruskal

adfs = [
    mim1_w1_adf,
    mim1_w5_adf,
    mim1_w25_adf,
    mim2_w1_adf,
    mim2_w5_adf,
    mim2_w25_adf,
    mim3_w1_adf,
    mim3_w5_adf,
    mim3_w25_adf,

    mim1_bfp015_adf,
    mim2_bfp015_adf,
    mim3_bfp015_adf,
    mim1_bfp03_adf,
    mim2_bfp03_adf,
    mim3_bfp03_adf,
    mim1_bfp06_adf,
    mim2_bfp06_adf,
    mim3_bfp06_adf,
    
    
    mim1_b2_adf,
    mim2_b2_adf,
    mim3_b2_adf,
    mim1_b10_adf,
    mim2_b10_adf,
    mim3_b10_adf,
]




for adf in adfs:
    print(stats.shapiro(adf[-num_agent:]))

ShapiroResult(statistic=0.6366536021232605, pvalue=0.0)
ShapiroResult(statistic=0.636650562286377, pvalue=0.0)
ShapiroResult(statistic=0.6366467475891113, pvalue=0.0)
ShapiroResult(statistic=0.636611819267273, pvalue=0.0)
ShapiroResult(statistic=0.636628270149231, pvalue=0.0)
ShapiroResult(statistic=0.6366214156150818, pvalue=0.0)
ShapiroResult(statistic=0.636647641658783, pvalue=0.0)
ShapiroResult(statistic=0.6366385221481323, pvalue=0.0)
ShapiroResult(statistic=0.6366386413574219, pvalue=0.0)
ShapiroResult(statistic=0.6366467475891113, pvalue=0.0)
ShapiroResult(statistic=0.6366214156150818, pvalue=0.0)
ShapiroResult(statistic=0.6366386413574219, pvalue=0.0)
ShapiroResult(statistic=0.6366491317749023, pvalue=0.0)
ShapiroResult(statistic=0.636618971824646, pvalue=0.0)
ShapiroResult(statistic=0.6366437673568726, pvalue=0.0)
ShapiroResult(statistic=0.6366915702819824, pvalue=0.0)
ShapiroResult(statistic=0.6366291046142578, pvalue=0.0)
ShapiroResult(statistic=0.6366828680038452, pvalue=0.

In [105]:
from pingouin import kruskal
from scipy.stats import mannwhitneyu

def epsilon_square(df1, df2):
    combined_df = pd.concat([df1, df2], axis=0)
    combined_df = combined_df.drop(columns=["index"])    
    kwTest = kruskal(data=combined_df, dv="opinion",between="op",detailed=True)
    kw_p = kwTest['p-unc'][0]
    kw_H = kwTest.H[0]
    n = combined_df.shape[0]
    es = kw_H * (n + 1)/(n**2 - 1)
    return kw_H, kw_p, es

def get_stats(df1, df2):
    kw_H, kw_p, es = epsilon_square(df1, df2)
    es = '{:.2f}'.format(es, 2)
    kw_H =  '{:.2f}'.format(kw_H, 2)
    if kw_p < 0.01:
        kw_p = '$<$ 0.01'
    else:
        kw_p = '{:.2f}'.format(kw_p, 2)    
    return kw_H, kw_p, es

## Kruskal-Wallis test

In [106]:
# mim1_adf = [
#     [mim1_w1_adf, mim1_w5_adf, mim1_w25_adf],
#     [mim1_bfp015_adf, mim1_bfp03_adf, mim1_bfp06_adf],
#     [mim1_b2_adf, mim1_b10_adf],
# ]

# mim2_adf = [
#     [mim2_w1_adf, mim2_w5_adf, mim2_w25_adf],
#     [mim2_bfp015_adf, mim2_bfp03_adf, mim2_bfp06_adf],
#     [mim2_b2_adf, mim2_b10_adf],
# ]


# mim3_adf = [
#     [mim3_w1_adf, mim3_w5_adf, mim3_w25_adf],
#     [mim3_bfp015_adf, mim3_bfp03_adf, mim3_bfp06_adf],
#     [mim3_b2_adf, mim3_b10_adf],
# ]


# mim_adf = [mim1_adf, mim2_adf, mim3_adf]

# exps = ["No Intervention", "Social Inoculation", "Account Banning"]
# bps = [r"$w$", r"$N_b$", r"$P_{bf}$"]
# comps = [[0, 1], [1, 2], [2, 0]]
# params = [[1,5,25], [0.15, 0.3, 0.6], [2, 10]]

# stats_table_path = "data/tables/kw_test_stats.csv"
# stats_table_latex_path = "data/tables/kw_test_stats.tex"

# open(stats_table_path, "w").close()

# with open(stats_table_path, "w") as fp:
#     table_header = r"Intervention, Bot Property, $param_1$, $param_2$, H Statistic, $p$ value, Effect Size"
#     fp.write(table_header + "\n")

# for i, adf in enumerate(mim_adf):
#     for j, sub_adf in enumerate(adf):
#         if j != 2:
#             for k, comp in enumerate(comps):
#                 # -----------------------------------> final
#                 sub_sub_adf_1 = sub_adf[comp[0]][-num_agent:]["opinion"]
#                 sub_sub_adf_2 = sub_adf[comp[1]][-num_agent:]["opinion"]
                
#                 kw_H, kw_p, es = get_stats(sub_sub_adf_1, sub_sub_adf_2)
                    
#                 with open(stats_table_path, "a") as fp:
#                     if k != 2:
#                         fp.write(f"{exps[i]}, {bps[j]}, {params[j][k]}, {params[j][k+1]}, {kw_H}, {kw_p}, {es}\n")
#                     else: 
#                         fp.write(f"{exps[i]}, {bps[j]}, {params[j][k]}, {params[j][0]}, {kw_H}, {kw_p}, {es}\n")
                        
#         else:
#             sub_sub_adf_1 = sub_adf[0][-num_agent:]["opinion"]
#             sub_sub_adf_2 = sub_adf[1][-num_agent:]["opinion"]
            
#             kw_H, kw_p, es = get_stats(sub_sub_adf_1, sub_sub_adf_2)
            
#             with open(stats_table_path, "a") as fp:
#                     fp.write(f"{exps[i]}, {bps[j]}, {params[j][0]}, {params[j][1]}, {kw_H}, {kw_p}, {es}\n")

# pd.read_csv(stats_table_path).to_latex(stats_table_latex_path, escape = False,index=False)

# THE FINAL STATISTICAL TEST

In [107]:
exps = ["No Intervention", "Account Banning", "Social Inoculation"]
bps = [r"$B_a$", r"$P_{bf}$", r"$N_b$"]
comps = [[0, 1], [0, 2]]
params = [[1,5,25], [0.15, 0.3, 0.6], [2, 10]]


w_adf = [
    [mim1_w1_adf, mim2_w1_adf, mim3_w1_adf],
    [mim1_w5_adf, mim2_w5_adf, mim3_w5_adf],
    [mim1_w25_adf, mim2_w25_adf, mim3_w25_adf],
]

bfp_adf = [
    [mim1_bfp015_adf, mim2_bfp015_adf, mim3_bfp015_adf],
    [mim1_bfp03_adf, mim2_bfp03_adf, mim3_bfp03_adf],
    [mim1_bfp06_adf, mim2_bfp06_adf, mim3_bfp06_adf],
]


b_adf = [
    [mim1_b2_adf, mim2_b2_adf, mim3_b2_adf],
    [mim1_b10_adf, mim2_b10_adf, mim3_b10_adf],
]

stats_table_path = "data/tables/kw_test_stats.csv"
stats_table_latex_path = "data/tables/kw_test_stats.tex"

open(stats_table_path, "w").close()

with open(stats_table_path, "w") as fp:
    table_header = r"Bot Property, Property Value, Interv. 1, Interv. 2, H Statistic, $p$ Value, Effect Size"
    fp.write(table_header + "\n")
    

mim_adf = [w_adf, bfp_adf, b_adf]

#--------------------------------- For loop --------------------------------

for i, adf in enumerate(mim_adf):
    for p, p_adf in enumerate(adf):
        for comp in comps:
            
            op_1 = p_adf[comp[0]][-num_agent:]["opinion"]
            op_2 = p_adf[comp[1]][-num_agent:]["opinion"]
            
            op_1 = op_1.reset_index()
            op_2 = op_2.reset_index()
            
            op_1['op'] = ['1'] * op_1.shape[0]
            op_2['op'] = ['2'] * op_2.shape[0]
            
                    
            kw_H, kw_p, es = get_stats(op_1, op_2)
            with open(stats_table_path, "a") as fp:
                fp.write(f"{bps[i]}, {params[i][p]}, {exps[comp[0]]}, {exps[comp[1]]}, {kw_H}, {kw_p}, {es}\n")
                
pd.read_csv(stats_table_path).to_latex(stats_table_latex_path, escape = False,index=False)


/tmp/ipykernel_31905/3248332066.py:57: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  pd.read_csv(stats_table_path).to_latex(stats_table_latex_path, escape = False,index=False)


In [108]:
from pingouin import kruskal, read_dataset
df = read_dataset('anova')
print(df)
kruskal(data=df, dv='Pain threshold', between='Hair color')

    Subject      Hair color  Pain threshold
0         1     Light Blond              62
1         2     Light Blond              60
2         3     Light Blond              71
3         4     Light Blond              55
4         5     Light Blond              48
5         6      Dark Blond              63
6         7      Dark Blond              57
7         8      Dark Blond              52
8         9      Dark Blond              41
9        10      Dark Blond              43
10       11  Light Brunette              42
11       12  Light Brunette              50
12       13  Light Brunette              41
13       14  Light Brunette              37
14       15   Dark Brunette              32
15       16   Dark Brunette              39
16       17   Dark Brunette              51
17       18   Dark Brunette              30
18       19   Dark Brunette              35


,Source,ddof1,H,p-unc
Kruskal,Hair color,3,10.58863,0.014172
